# Welcome to the Lab 🥼🧪
## How have YoY Rental Prices changed for all zipcodes in Florida?


Welcome to the Lab - LFG

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/inspiration/map_yoy_rental_rates_by_zip.ipynb)


You can expect to create this:

![Chart](../assets/fl_yoy_rental_prices.png)

In [ ]:
# Environment setup
import os
import sys
import json
import subprocess
from datetime import datetime
from urllib.request import urlopen

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [ ]:
import parcllabs
import pandas as pd
import plotly.express as px
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
client = ParclLabsClient(api_key=api_key)

In [ ]:
# lets get all metros in the country
zips = client.search_markets.retrieve(
    location_type='ZIP5',
    state_abbreviation='FL',
    as_dataframe=True,
    params={
        'limit': 1000
    },
    auto_paginate=True
)

In [ ]:
# get the ids
fl_zips = zips['parcl_id'].tolist()
len(fl_zips)

In [ ]:
# How big are the investors? Let's find out
prices = client.market_metrics_housing_event_prices.retrieve_many(
    parcl_ids=fl_zips,
    as_dataframe=True,
    params={
        'limit': 13
    }
)

In [ ]:
# merge with search results to get geoid
df = pd.merge(prices, zips, on='parcl_id')

In [ ]:
# lets calculate YoY rents
col_name = 'price_median_new_rental_listings'
price = df[['date', 'parcl_id', 'name', col_name]]
last = price.loc[price['date'] == '2024-03-01']
first = price.loc[price['date'] == '2023-03-01']

last = last.loc[last[col_name].notnull()]
first

In [ ]:
# drop null records
first = first.loc[first[col_name].notnull()]
first = first.rename(columns={col_name: f'{col_name}_2023'})
first

In [ ]:
out = pd.merge(last[['parcl_id', 'name', col_name]], first[['parcl_id', f'{col_name}_2023']], on='parcl_id')

out['yoy_pct_change'] = (out[col_name] - out[f'{col_name}_2023'])/out[f'{col_name}_2023']
out['yoy_pct_change'] = out['yoy_pct_change']*100
out = out.rename(columns={'name': 'ZCTA5CE10'})
out.head()

In [ ]:
out.sort_values('yoy_pct_change', ascending=False)

In [ ]:
# lets create the map
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/fl_florida_zip_codes_geo.min.json') as response:
    zips = json.load(response)

In [ ]:
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white+(1).svg'
}

# set charting constants
labs_logo_dict = dict(
        source=labs_logo_lookup['white'],
        xref="paper",
        yref="paper",
        x=0.5,  # Centering the logo below the title
        y=1.02,  # Adjust this value to position the logo just below the title
        sizex=0.15, 
        sizey=0.15,
        xanchor="center",
        yanchor="bottom"
)

media_img_size_lookup = {
    'X': {
        'width': 1600,
        'height': 900
    }
}

PLATFORM = 'X'

# set image sizes
IMG_WIDTH = media_img_size_lookup[PLATFORM]['width']
IMG_HEIGHT = media_img_size_lookup[PLATFORM]['height']

# plotting title settings
PLOT_TITLE_SETTINGS = {
        'y':0,
        'x':0,
        'xanchor': 'left',
        'yanchor': 'bottom'
    }

In [ ]:
import plotly.express as px
import pandas as pd

# Assuming 'df' and 'ga_zips' are properly configured

# Define the color scale
color_continuous_scale=[
    (0, "#FF0000"),
    (0.5, "#EEF7FF"),  
    (1, "#00FF00") 
]

# Set your Mapbox access token
mapbox_access_token = os.getenv('MBOX_ACCESS_TOKEN')

px.set_mapbox_access_token(mapbox_access_token)

# Create the choropleth map
fig = px.choropleth_mapbox(out,
                           geojson=zips, 
                           featureidkey='properties.ZCTA5CE10',
                           locations='ZCTA5CE10',
                           color='yoy_pct_change',
                           color_continuous_scale=color_continuous_scale,
                           range_color=(-10, 10),
                           mapbox_style="mapbox://styles/mapbox/dark-v9",
                           zoom=6,
                           # center={"lat": 32.3846, "lon": -83.2554},# latitude 32° 38´ 46" N., longitude 83° 25´ 54"
                           opacity=0.9,
                           labels={'yoy_pct_change':'YoY % Change'}
                          )

# Adjust color bar and title text color
fig.update_coloraxes(colorbar=dict(
    title="% Change",
    tickfont=dict(color='white', size=8),
    title_font=dict(color='white', size=10),  # Title font size set to 10
    # modify color bar position to the bottom center
    #x=0.5,  # Color bar x position (left)
    y=-0.01, # Color bar y position (middle vertically)
    len=0.35, # Length of the color bar (70% of the vertical dimension)
    orientation='h',  # Horizontal color bar
    thickness=15,  # Thickness of the color bar
    xanchor='center',  # Center the color bar horizontally
)
)


fig.add_layout_image(dict(
     source=labs_logo_lookup['white'],
     xref="paper",
     yref="paper",
     x=0.5,
     y=1.03,
     sizex=0.15, 
     sizey=0.15,
     xanchor="center",
     yanchor="top"
))

# Update the layout for background color, margins, and figure dimensions
fig.update_layout(
    title='YoY Asking Rent Price Changes, Florida',
    title_x=0.5,  # Center the title
    title_font=dict(color='white', size=16),
    paper_bgcolor='#080D16',
    geo_bgcolor='#080D16',
    margin={"r":0, "t":60, "l":0, "b":0},  # Adjust top margin to give space for the title
    width=1200,
    height=800,
    

)

# Display the map
fig.show()